![](./assets-resources/embedding-search.png)

In [1]:
from langchain.document_loaders.csv_loader import CSVLoader

In [2]:
loader = CSVLoader(file_path="./prompts.csv")
data = loader.load_and_split()[:20]
data

[Document(page_content='act: Linux Terminal\nprompt: I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd', metadata={'source': './prompts.csv', 'row': 0}),
 Document(page_content='act: English Translator and Improver\nprompt: I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning same, but 

In [4]:
import pandas as pd

df = pd.read_csv("./prompts.csv")

df.head()

,act,prompt
0,Linux Terminal,I want you to act as a linux terminal. I will ...
1,English Translator and Improver,"I want you to act as an English translator, sp..."
2,`position` Interviewer,I want you to act as an interviewer. I will be...
3,JavaScript Console,I want you to act as a javascript console. I w...
4,Excel Sheet,I want you to act as a text based excel. you'l...


In [5]:
from IPython.display import Markdown

Markdown(data[0].page_content)

act: Linux Terminal
prompt: I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd

In [6]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="llama3",
    verbose=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
) # allow for streaming the response

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100) # you should experiment with!
all_splits = text_splitter.split_documents(data)

In [8]:
all_splits

[Document(page_content='act: Linux Terminal\nprompt: I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd', metadata={'source': './prompts.csv', 'row': 0}),
 Document(page_content='act: English Translator and Improver\nprompt: I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning same, but 

In [19]:
# Embed and store
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings  # We can also try Ollama embeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OllamaEmbeddings(model="llama3"),persist_directory="new-dir")

In [20]:
question = "Prompt screenwriting?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [21]:
docs

[Document(page_content='act: Debater\nprompt: I want you to act as a debater. I will provide you with some topics related to current events and your task is to research both sides of the debates, present valid arguments for each side, refute opposing points of view, and draw persuasive conclusions based on evidence. Your goal is to help people come away from the discussion with increased knowledge and insight into the topic at hand. My first request is "I want an opinion piece about Deno."', metadata={'row': 16, 'source': './prompts.csv'}),
 Document(page_content='act: Advertiser\nprompt: I want you to act as an advertiser. You will create a campaign to promote a product or service of your choice. You will choose a target audience, develop key messages and slogans, select the media channels for promotion, and decide on any additional activities needed to reach your goals. My first suggestion request is "I need help creating an advertising campaign for a new type of energy drink targeti

In [22]:
Markdown(docs[0].page_content)

act: Debater
prompt: I want you to act as a debater. I will provide you with some topics related to current events and your task is to research both sides of the debates, present valid arguments for each side, refute opposing points of view, and draw persuasive conclusions based on evidence. Your goal is to help people come away from the discussion with increased knowledge and insight into the topic at hand. My first request is "I want an opinion piece about Deno."

In [23]:
Markdown(docs[1].page_content)

act: Advertiser
prompt: I want you to act as an advertiser. You will create a campaign to promote a product or service of your choice. You will choose a target audience, develop key messages and slogans, select the media channels for promotion, and decide on any additional activities needed to reach your goals. My first suggestion request is "I need help creating an advertising campaign for a new type of energy drink targeting young adults aged 18-30."

In [24]:
Markdown(docs[2].page_content)

act: Screenwriter
prompt: I want you to act as a screenwriter. You will develop an engaging and creative script for either a feature length film, or a Web Series that can captivate its viewers. Start with coming up with interesting characters, the setting of the story, dialogues between the characters etc. Once your character development is complete - create an exciting storyline filled with twists and turns that keeps the viewers in suspense until the end. My first request is "I need to write a romantic drama movie set in Paris."

In [25]:
Markdown(docs[3].page_content)

act: Debate Coach
prompt: I want you to act as a debate coach. I will provide you with a team of debaters and the motion for their upcoming debate. Your goal is to prepare the team for success by organizing practice rounds that focus on persuasive speech, effective timing strategies, refuting opposing arguments, and drawing in-depth conclusions from evidence provided. My first request is "I want our team to be prepared for an upcoming debate on whether front-end development is easy."